In [1]:
# File: market_maker/cost.py

import numpy as np

def lmsr_cost(q: np.ndarray, b: float = 1.0) -> float:
    """
    Total cost C(q) of the current market state under LMSR.
    q: outstanding share counts per bucket
    b: liquidity parameter
    """
    return b * np.log(np.sum(np.exp(q / b)))

def share_cost(q: np.ndarray, bucket_index: int, b: float = 1.0) -> float:
    """
    Cost to buy one additional share in the specified bucket.
    q: outstanding share counts per bucket
    bucket_index: which bucket to buy in
    b: liquidity parameter
    """
    q = q.astype(float)
    delta = np.zeros_like(q)
    delta[bucket_index] = 1.0
    return lmsr_cost(q + delta, b) - lmsr_cost(q, b)


In [2]:
def _capped_normal_weights(q: np.ndarray, center: int, width: int, sigma: float) -> np.ndarray:
    """
    Discrete, normalized Gaussian weights over buckets [center-width, center+width].
    """
    n = q.size
    idx = np.arange(n)
    mask = np.abs(idx - center) <= width
    w = np.zeros(n, dtype=float)
    w[mask] = np.exp(-0.5 * ((idx[mask] - center) / sigma) ** 2)
    return w / w.sum()

def max_shares_with_budget(
    q: np.ndarray,
    budget: float,
    center: int,
    width: int = 5,
    sigma: float = 1.0,
    b: float = 1.0,
    tol: float = 1e-8,
    max_iter: int = 50,
) -> np.ndarray:
    """
    Return how many shares per bucket can be bought with `budget` units,
    following a capped normal distribution around `center` ± `width`.

    q       : current outstanding shares (length N)
    budget  : total units available to spend
    center  : index of middle bucket
    width   : half‑width of purchase interval
    sigma   : standard deviation of Gaussian shape
    b       : LMSR liquidity parameter
    tol     : binary search precision
    max_iter: binary search iterations

    Returns an array Δq of length N (shares to buy per bucket).
    """
    w = _capped_normal_weights(q, center, width, sigma)

    def total_cost(alpha: float) -> float:
        return lmsr_cost(q + alpha * w, b) - lmsr_cost(q, b)

    # exponential search for upper bound
    lo, hi = 0.0, 1.0
    while total_cost(hi) <= budget:
        hi *= 2

    # binary search
    for _ in range(max_iter):
        mid = (lo + hi) / 2
        if total_cost(mid) <= budget:
            lo = mid
        else:
            hi = mid

    return lo * w


In [6]:
def simulate_two_traders(
    N: int = 100,
    budget1: float = 100.0,
    budget2: float = 100.0,
    center: int = 50,
    width: int = 5,
    sigma: float = 1.0,
    b: float = 1.0
) -> None:
    """
    Simulate two traders each buying a capped normal distribution
    around `center` ± `width` with their respective budgets.
    Prints the number of shares bought by each trader in each bucket.
    """
    # Initial market state
    q = np.zeros(N)

    # First trader
    delta1 = max_shares_with_budget(q, budget1, center, width, sigma, b)
    q += delta1

    # Second trader
    delta2 = max_shares_with_budget(q, budget2, center, width, sigma, b)

    # Print results
    print(f"Trader 1 (budget={budget1}):")
    print("Bucket | Shares Bought")
    print("----------------------")
    for i in range(center - width, center + width + 1):
        print(f"{i:6d} | {delta1[i]:.4f}")

    print(f"\nTrader 2 (budget={budget2}):")
    print("Bucket | Shares Bought")
    print("----------------------")
    for i in range(center - width, center + width + 1):
        print(f"{i:6d} | {delta2[i]:.4f}")

if __name__ == "__main__":
    simulate_two_traders()


Trader 1 (budget=100.0):
Bucket | Shares Bought
----------------------
    45 | 0.0004
    46 | 0.0351
    47 | 1.1621
    48 | 14.1568
    49 | 63.4462
    50 | 104.6052
    51 | 63.4462
    52 | 14.1568
    53 | 1.1621
    54 | 0.0351
    55 | 0.0004

Trader 2 (budget=100.0):
Bucket | Shares Bought
----------------------
    45 | 0.0004
    46 | 0.0335
    47 | 1.1109
    48 | 13.5335
    49 | 60.6531
    50 | 100.0000
    51 | 60.6531
    52 | 13.5335
    53 | 1.1109
    54 | 0.0335
    55 | 0.0004
